In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [2]:
import cv2
import numpy as np
from sklearn import svm
import time

bin_n = 16 # Number of bins

def Rotation(img) : 
    thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    
    ## 0보다 큰 모든 픽셀 값의 (x,y) 좌표를 구한다.
    ## 그리고 이 좌표들을 rotated 연산을 하는데 사용한다.
    coords = np.column_stack(np.where(thresh>0))
    
    ## cv2.minAreaRect : 모양에 외접하면서 면적이 가장 작은 직사각형을 구한다.
    ## 그 좌표가 주어진 마지막 요소는 angle로 사용
    angle = cv2.minAreaRect(coords)[-1]
    
    if angle < -45 :
        angle = -(90 + angle)
    else :
        angle = -angle
    
    h,w = img.shape[:2]
    center = (w//2, h//2)
    
    ## Affine Transform을 위한 Matrix 구하기
    ## 1.0으로 scale 유지
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    
    ## 보간법으로 Cubic 사용
    ## cv2.BORDER_REPLICATE : 원본의 가장 밖의 테투리에서의 행이나 열은 추가 테두리로 복제된다.
    rotated = cv2.warpAffine(img, M, (w,h), flags = cv2.INTER_CUBIC, borderMode = cv2.BORDER_REPLICATE)
    
    return rotated

def HOG(img):
    gx = cv2.Sobel(img, cv2.CV_32F, 1, 0)   ## x, y방향으로 Gradient를 구한다.
    gy = cv2.Sobel(img, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy)      ## magnitude와 angle값을 구하는 식이 있으나 라이브러리로 한번에 처리한다.
    bins = np.int32(bin_n*ang/(2*np.pi))    ## 360도를 16개 구간으로 나누었으니, ang값이 16개 구간 중 어디에 속하는지 계산한다.
    
    ## 28x28 orientation map을 4등분 한다. ( 1개 block = 14x14 pixel , 4개의 7x7 cell )
    bin_cells = bins[:14,:14], bins[14:,:14], bins[:14,14:], bins[14:,14:]
    mag_cells = mag[:14,:14], mag[14:,:14], mag[:14,14:], mag[14:,14:]
    
    ## 각각의 block의 orientation을 구함
    hists = [np.bincount(b.ravel(), m.ravel(), bin_n) for b, m in zip(bin_cells, mag_cells)]
    
    ## 각 block의 Orientation을 이어 붙여서 descriptor로 만든다.
    hist = np.hstack(hists)     # hist is a 64 bit vector
    
    return hist

if __name__ == "__main__":
    start_time = time.time()
    # Train 데이터와 Test 데이터 나누기
    split_ratio = 0.8
    n_train = int(split_ratio * mnist.data.shape[0])
    
    train_set = mnist.data[:n_train]
    test_set = mnist.data[n_train:]
    
    ## 각각의 training 이미지를 정방향으로 Ratation
    rotated = []
    for i in range(0, len(train_set)) :
        rotated.append( Rotation(np.array(train_set[i].reshape(28,28), np.uint8)) )
        
    ## Rotation한 이미지로 HOG Descriptor 계산
    hogdata = []
    for i in range(0, len(rotated)) :
        hogdata.append( HOG(np.array(rotated[i].reshape(28,28), np.uint8)) )
    
    ## HOG를 거쳐서 나온 descriptor와 각각의 Training Data의 Label로 SVM 지도학습 수행
    ## SVM을 사용하여 지도학습을 수행할 경우에 Data set을 비선형으로 분류할지, 선형으로 분류할지 정해야한다.
    ## 그러기 위해서는 자신의 Data set을 좌표평면에 뿌려 확인하는 방법이 필요할 것 같다.
    ## kernel = linear(선형), kernel = rbf (비선형)
    trainData = np.float32(hogdata).reshape(-1,64)
    trainLabel = mnist.target[:n_train]
    
    clf = svm.SVC(kernel='rbf')
    clf.fit(trainData, trainLabel)
    
    ## 위에서 수행한 지도학습 모델에 input값으로 Test 이미지의 Hog Descriptor를 넣어서 이미지의 그려진 숫자 예측
    rotated = []
    for i in range(0, len(test_set)) :
        rotated.append( Rotation(np.array(test_set[i].reshape(28,28), np.uint8)) )
    
    hogdata = []
    for i in range(0, len(rotated)) :
        hogdata.append( HOG(np.array(rotated[i].reshape(28,28), np.uint8)) )
        
    testData = np.float32(hogdata).reshape(-1,64)
    result = clf.predict(testData)
    
    end_time = time.time()
    print("Total_Time: {} sec".format(end_time-start_time))
    
    ## 정확도 검사
    testLabel = mnist.target[n_train:]
    correct = 0
    for i in range(0, len(result)) :
        if result[i] == testLabel[i] :
            correct += 1
    accuracy = 100 * correct/len(result)
    print('accuracyacy : %lf %%' %accuracy)
    
    ## Test 이미지 분류 값 출력
    testImage_value = clf.predict(testData[129].reshape(1,-1))
    print('Test 이미지 분류 값 : %s' % testImage_value[0])
    
    cv2.imshow('Test Image', test_set[129].reshape(28,28))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Total_Time: 134.98919653892517 sec
accuracyacy : 96.985714 %
Test 이미지 분류 값 : 0
